In [1]:
import os

file_path = "../upload_folder/product_a/electrolux_washing_machine.pdf"
# file_path = "../data_folder/product_b/lg_washing_machine.pdf"
# file_path = "../data_folder/product_a_small/electrolux_washing_machine.pdf"

# load the pdf to image
folder_path = os.path.dirname(file_path)
preprocessed_data_folder = folder_path.replace("upload_folder", "preprocessed_data")
# product id is folder_path folder name

product_id = os.path.basename(folder_path)
persist_directory = f"../db_folder/{product_id}"
persist_directory

'../db_folder/product_a'

In [2]:
preprocessed_data_folder

'../preprocessed_data/product_a'

## Can skip step #0 if preprocessed_data have text file already


### step 0


In [2]:
# from pdf2image import convert_from_path
# import pdfplumber


# os.makedirs(preprocessed_data_folder, exist_ok=True)

# image_name_list = []


# def pdf_save_text_each_page(file_path):
#     try:
#         with pdfplumber.open(file_path) as pdf:
#             # Read all pages and extract text
#             for ind, page in enumerate(pdf.pages):
#                 text = page.extract_text()
#                 # save text to image_{ind}.txt
#                 image_name = f"image_{ind}.txt"
#                 image_name_list.append(image_name)
#                 file_path = os.path.join(preprocessed_data_folder, image_name)
#                 with open(file_path, "w") as f:
#                     f.write(text)

#     except Exception as e:
#         print("Error reading PDF:", e)


# pdf_save_text_each_page(file_path)

### step 1


In [3]:
image_name_list = os.listdir(preprocessed_data_folder)
# .txt only

image_name_list = [i for i in image_name_list if i.endswith(".txt")]
import re


def extract_numeric_part(filename):
    # Use regular expression to extract the numeric part of the filename
    match = re.search(r"\d+", filename)
    if match:
        return int(match.group())
    return None


# Sort the file names based on the numeric part
image_name_list = sorted(image_name_list, key=extract_numeric_part)

print(image_name_list[:3])
print(image_name_list[-3:])

['image_0.txt', 'image_1.txt', 'image_2.txt']
['image_37.txt', 'image_38.txt', 'image_39.txt']


In [5]:
from transformers import GPT2TokenizerFast
from langchain.text_splitter import RecursiveCharacterTextSplitter

/Users/user/miniconda3/envs/chatgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [6]:
image_name_list[:3]

['image_0.txt', 'image_1.txt', 'image_2.txt']

In [7]:
meta_data_list = []
text_list = []

for ind, image_name in enumerate(image_name_list):
    text_path = os.path.join(
        preprocessed_data_folder, image_name.split(".")[0] + ".txt"
    )

    with open(text_path, "r") as f:
        text = f.read()

    meta_data = {
        "page_index": ind + 1,
        "data_name": file_path.split("/")[-1],
    }
    # meta_data = {"hellow": ind+1}
    text_list.append(text)
    meta_data_list.append(meta_data)
# Step 3: Create function to count tokens
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")


def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

In [8]:
# Step 4: Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    # chunk_size=300,
    chunk_size=800,
    chunk_overlap=100,
    length_function=count_tokens,
)
chunks = text_splitter.create_documents(text_list, meta_data_list)

In [9]:
len(chunks)

56

In [11]:
text_list = []
meta_data_list = []
for chunk in chunks:
    text_list.append(chunk.page_content)
    meta_data_list.append(chunk.metadata)

In [12]:
from langchain.embeddings import OpenAIEmbeddings
from chromadb.utils import embedding_functions

import os
import openai
from dotenv import load_dotenv
import pandas as pd

credential_path = "../../.credential"
assert os.path.exists(credential_path)
load_dotenv(credential_path)

openai.api_key = os.environ["OPENAI_API_KEY"]

In [13]:
# client = chromadb.Client(chroma_setting)
from chromadb import HttpClient

client = HttpClient()

In [14]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY", "missing openai api key"),
    model_name="text-embedding-ada-002",
)
collection = client.get_or_create_collection(
    name="product_a_small", embedding_function=openai_ef
)
# collection = client.delete_collection(
#     name="product_a"
# )
# collection = client.create_collection(
#     name="product_a", embedding_function=openai_ef
# )

In [15]:
# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=text_list,  # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=meta_data_list,  # filter on these!
    ids=[f"doc_{i+1}" for i in range(len(text_list))],  # unique for each doc
)

In [16]:
len(text_list)

56

In [17]:
import requests

url = "http://localhost:8000/api/v1/collections"
reponse = requests.get(url)
reponse.json()

[{'name': 'product_a_small',
  'id': '17f5ae88-3bee-4ede-add9-034a832e5454',
  'metadata': None}]

In [18]:
for collection_data in reponse.json():
    if collection_data["name"] == "product_a_small":
        collection_id_product_a = collection_data["id"]
        break
collection_id_product_a

'17f5ae88-3bee-4ede-add9-034a832e5454'

## this last cell should print

{'ids': ['doc_1',
'doc_2',
'doc_3',
'doc_4',
'doc_5',
...


In [20]:
collection_id = collection_id_product_a
# collection_id = "b2c4031e-4ade-40cf-ad4e-12fb9cb5fa4c"
url = f"http://localhost:8000/api/v1/collections/{collection_id}/get"
reponse = requests.post(url, json={})
reponse.json()

{'ids': ['doc_1',
  'doc_2',
  'doc_3',
  'doc_4',
  'doc_5',
  'doc_6',
  'doc_7',
  'doc_8',
  'doc_9',
  'doc_10',
  'doc_11',
  'doc_12',
  'doc_13',
  'doc_14',
  'doc_15',
  'doc_16',
  'doc_17',
  'doc_18',
  'doc_19',
  'doc_20',
  'doc_21',
  'doc_22',
  'doc_23',
  'doc_24',
  'doc_25',
  'doc_26',
  'doc_27',
  'doc_28',
  'doc_29',
  'doc_30',
  'doc_31',
  'doc_32',
  'doc_33',
  'doc_34',
  'doc_35',
  'doc_36',
  'doc_37',
  'doc_38',
  'doc_39',
  'doc_40',
  'doc_41',
  'doc_42',
  'doc_43',
  'doc_44',
  'doc_45',
  'doc_46',
  'doc_47',
  'doc_48',
  'doc_49',
  'doc_50',
  'doc_51',
  'doc_52',
  'doc_53',
  'doc_54',
  'doc_55',
  'doc_56'],
 'embeddings': None,
 'metadatas': [{'data_name': 'electrolux_washing_machine.pdf',
   'page_index': 1},
  {'data_name': 'electrolux_washing_machine.pdf', 'page_index': 2},
  {'data_name': 'electrolux_washing_machine.pdf', 'page_index': 2},
  {'data_name': 'electrolux_washing_machine.pdf', 'page_index': 3},
  {'data_name': 'ele